In [1]:
# 1. 표준 라이브러리
import sys
from pathlib import Path
import platform
import time
import json
import os
import joblib
from datetime import datetime
from typing import Optional
from dataclasses import dataclass


# 2. 서드파티 라이브러리 

# 2-1. 시각화
import plotly.express as px

import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm
import seaborn as sns

# 2-2. 
import shap
from imblearn.pipeline import Pipeline as ImbPipeline
from lightgbm import LGBMClassifier, early_stopping, log_evaluation
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, train_test_split
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score, make_scorer, recall_score, 
    precision_score, f1_score, fbeta_score, average_precision_score, balanced_accuracy_score, precision_recall_fscore_support
)
from scipy.stats import uniform, randint
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.base import BaseEstimator

sys.path.append(str(Path.cwd().parent))
from utils import DATA_DIR, MODEL_DIR

In [2]:
# 한글 폰트 설정
if platform.system() == 'Windows':
    plt.rcParams['font.family'] = 'Malgun Gothic'
elif platform.system() == 'Darwin':  # macOS
    plt.rcParams['font.family'] = 'AppleGothic'
else:  # Linux
    plt.rcParams['font.family'] = 'NanumGothic'

plt.rcParams['axes.unicode_minus'] = False

mpl.rcParams['axes.unicode_minus'] = False
# 폰트 개인 경로에 맞춰서 변경
# FONT_DIR = Path("/path/to/fonts")
# font_path = FONT_DIR / 'FREESENTATION-6SEMIBOLD.ttf'
# prop = fm.FontProperties(fname=font_path)

In [3]:
pandas_kwargs = {
    'parse_dates': ['검정일자'],
    'date_format': '%Y-%m-%d'
}

milk: pd.DataFrame = pd.read_csv(DATA_DIR /'interim' / 'milk.csv', **pandas_kwargs)
milk.head()

,농장아이디,개체번호,검정일자,누적착유일(연계),무지고형분,MUN,305일유량,305일무지고형분,전산차비유지속성,전산차건유전유량,...,농장구분,정액코드분류,가격,가격미달,가격구분,분만간격,분만월령,공태일수_log,계절_sin,비유단계_sin
0,20249,20120709020022,2020-01-11,154,0,1,9714.0,805.0,50.5,16.1,...,1,NaN,1056.34,0,1,NaN,85,NaN,-2.449294e-16,1.000000e+00
1,20249,20120504020095,2020-01-11,52,0,1,NaN,NaN,68.8,22.6,...,1,NaN,1056.34,0,1,NaN,90,NaN,-2.449294e-16,1.000000e+00
2,20249,20111008020210,2020-01-11,115,0,1,11008.0,971.0,54.1,24.5,...,1,NaN,1060.34,0,1,NaN,95,NaN,-2.449294e-16,1.224647e-16
3,20249,20121014020049,2020-01-11,290,1,1,11318.0,1011.0,58.6,37.8,...,1,7HO,1062.31,0,1,NaN,77,5.4161,-2.449294e-16,-1.000000e+00
4,20249,20130812020216,2020-01-11,100,0,1,11324.0,957.0,68.1,20.5,...,1,NaN,1056.34,0,1,NaN,75,NaN,-2.449294e-16,1.224647e-16


In [4]:
df = milk

features = [
    '전산차건유전유량', 
    '산차', 
    '농후사료급여량(연계)', 
    '공태일수', 
    '계절', 
    '농장구분', 
    '분만간격',
    # '정액코드분류'
]

target = '가격미달'

In [7]:
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

from tabpfn import TabPFNClassifier
from tabpfn.constants import ModelVersion

# Load data
split_date = '2021-08-01'
train = df[df['검정일자'] < split_date]
test = df[df['검정일자'] >= split_date]

X_train, X_test = train[features], test[features]
y_train, y_test = train[target], test[target]

# Initialize a classifier
# clf = TabPFNClassifier()  # Uses TabPFN 2.5 weights, finetuned on real data.
# To use TabPFN v2:
clf = TabPFNClassifier.create_default_for_version(ModelVersion.V2, device='cuda', ignore_pretraining_limits=True)
clf.fit(X_train, y_train)


# Predict probabilities
prediction_probabilities = clf.predict_proba(X_test)
print("ROC AUC:", roc_auc_score(y_test, prediction_probabilities[:, 1]))

# Predict labels
predictions = clf.predict(X_test)
print("Accuracy", accuracy_score(y_test, predictions))

ROC AUC: 0.6426465513125679
Accuracy 0.8860462387043774


In [8]:
pr_auc = average_precision_score(y_test, prediction_probabilities[:, 1])
print(f"PR-AUC (Test Set): {pr_auc:.4f}")

PR-AUC (Test Set): 0.1981
